#### Find the files that isn't downloaded

In [1]:
import os

In [2]:
files_expected = [f"shhs1-{i}_cleaned.parquet" for i in range(200001, 200501)]
for folder, subfolder, files in os.walk("data/shhs/polysomnography/edfs/shhs1/"):
    for file in files:
        if  file.endswith(".parquet"):
            files_expected.remove(file)

print(' '.join([file.split(".")[0].split("-")[-1].split("_")[0] for file in files_expected]))

200033 200069 200125 200142 200146 200151 200152 200153 200154 200155 200156 200157 200158 200159 200160 200161 200162 200163 200164 200165 200166 200167 200168 200169 200170 200171 200172 200173 200174 200175 200176 200177 200178 200179 200180 200181 200182 200183 200184 200185 200186 200187 200188 200189 200190 200191 200192 200193 200194 200195 200196 200197 200198 200199 200200 200201 200202 200203 200204 200205 200206 200207 200208 200209 200210 200211 200212 200213 200214 200215 200216 200217 200218 200219 200220 200221 200222 200223 200224 200225 200226 200227 200228 200229 200230 200231 200232 200233 200234 200235 200236 200237 200238 200239 200240 200241 200242 200243 200244 200245 200246 200247 200248 200249 200250 200251 200252 200253 200254 200255 200256 200257 200258 200259 200260 200261 200262 200263 200264 200265 200266 200267 200268 200269 200270 200271 200272 200273 200274 200275 200276 200277 200278 200279 200280 200281 200282 200283 200284 200285 200286 200287 200288

In [3]:
len(files_expected)

355

#### Print Directory Structure

In [4]:
import os

def print_tree(root, prefix=""):
    files = sorted(os.listdir(root))
    entries = [f for f in files if not f.startswith('.')]  # skip hidden files
    for index, name in enumerate(entries):
        path = os.path.join(root, name)
        is_last = index == len(entries) - 1
        connector = "└── " if is_last else "├── "
        print(prefix + connector + name)

        if os.path.isdir(path):
            extension = "    " if is_last else "│   "
            print_tree(path, prefix + extension)

# Example usage
print_tree("../")

├── LICENSE
├── README.md
├── sleep-project
│   ├── README.md
│   ├── dist
│   │   ├── sleepdataspo2-0.1.0-py3-none-any.whl
│   │   └── sleepdataspo2-0.1.0.tar.gz
│   ├── pyproject.toml
│   ├── sleepdataspo2
│   │   ├── __init__.py
│   │   ├── clean_features.py
│   │   ├── constants.py
│   │   ├── download_data.py
│   │   ├── engineer_features.py
│   │   ├── load_data.py
│   │   ├── plot_graphs.py
│   │   ├── process.py
│   │   └── run_pipeline.py
│   └── sleepdataspo2.egg-info
│       ├── PKG-INFO
│       ├── SOURCES.txt
│       ├── dependency_links.txt
│       ├── requires.txt
│       └── top_level.txt
└── usage
    ├── INSTRUCTIONS.md
    ├── README.md
    ├── cert.pem
    ├── cfs-visit5-800045.edf
    ├── cfs-visit5-dataset-0.7.0.csv
    ├── data
    │   ├── cfs
    │   │   ├── images
    │   │   │   ├── cleaned
    │   │   │   │   └── cfs-visit5-800045.png
    │   │   │   └── original
    │   │   │       └── cfs-visit5-800045.png
    │   │   └── polysomnography
    │   │       └──

### What `sleepdataspo2` does?

1. Drop non-psysiological values (below 50 and above 100)
2. Remove sharp jumps (8% quick drop or rise)
3. Smooth with median filter with window = ($spo2[i] = median(spo2[i-int(\frac{9}{2}): i+int(\frac{9}{2})+1])$)
4. Remove block artifacts (extended periods of invalid, flat, or low-quality signal, often due to sensor issues rather than true physiological events)
5. Downsample to 1Hz
6. Replace NAN by interpolating
7. Saves the cleaned SpO2 signals for later use (with the images of originaland and cleaned signals)
8. Extract the features described [here](https://static-content.springer.com/esm/art%3A10.1038%2Fs41467-023-40604-3/MediaObjects/41467_2023_40604_MOESM1_ESM.pdf) just before last 2-3 pages and save it

In [5]:
import numpy as np

In [6]:
np.nan > 10

False

In [7]:
import pandas as pd
df = pd.read_parquet("data/shhs/polysomnography/edfs/shhs1/shhs1-200002_cleaned.parquet")

In [8]:
df.head()

,SaO2
time,
0,96.0
1,96.0
2,96.0
3,96.0
4,97.0


In [9]:
spo2 = df.SaO2.to_numpy()

In [10]:
from pobm.obm.desat import DesaturationsMeasures
from pobm.obm.burden import HypoxicBurdenMeasures
from pobm.obm.complex import ComplexityMeasures
from pobm.obm.general import OverallGeneralMeasures
from pobm.obm.periodicity import  PRSAMeasures, PSDMeasures
from pobm._ResultsClasses import DesatMethodEnum
from colorama import Fore, Style

light_green = Fore.LIGHTGREEN_EX
reset = Style.RESET_ALL

### Statistical Features

| #  | **Feature**       | **Category**       | **Description**                                                         |
| -- | ----------------- | ------------------ | ----------------------------------------------------------------------- |
| 1  | AV                | General Statistics | Mean SpO₂ over the entire signal                                        |
| 2  | MED               | General Statistics | Median SpO₂                                                             |
| 3  | Min               | General Statistics | Minimum SpO₂ value                                                      |
| 4  | SD                | General Statistics | Standard deviation of SpO₂                                              |
| 5  | RG                | General Statistics | Range (max - min) of SpO₂                                               |
| 6  | Px                | General Statistics | xth percentile of SpO₂ values                                           |
| 7  | Mx                | General Statistics | Percentage of signal below median oxygen saturation                     |
| 8  | ZCx               | General Statistics | Number of zero-crossings at x% level (threshold crossing count)         |
| 9  | ∆Ix (Delta Index) | General Statistics | Mean difference between consecutive SpO₂ values                         |

In [11]:
statistics_class = OverallGeneralMeasures(ZC_Baseline=90, percentile=1, M_Threshold=2, DI_Window=12)
# Compute the biomarkers
print(f"{light_green}[DEBUG]{reset} Computing statistical features")
results_statistics = statistics_class.compute(spo2)
print(f"{light_green}[DEBUG]{reset} Completed computing statistical features")

[DEBUG] Computing statistical features
[DEBUG] Completed computing statistical features


In [12]:
results_statistics

OverallGeneralMeasuresResult(AV=93.98775793650793, MED=94.0, Min=89.0, SD=1.9385556430364006, RG=10.0, P=89.0, M=10.773809523809524, ZC=1067, DI=1.1042104129350758, K=-0.03367820234715602, SK=-0.5076587022579885, MAD=1.4826)

### Periodicity Features

PRSA stands for Phase-Rectified Signal Averaging — a signal processing technique used to detect and quantify quasi-periodic oscillations in noisy, nonstationary physiological signals, like SpO₂, heart rate, or PPG.

PRSA enhances weak periodic patterns that might be hidden by noise. It’s especially useful in medical signals where regular patterns (like desaturations) don’t occur at perfectly regular intervals, but still follow a quasi-periodic trend.

| #  | **Feature**       | **Category**       | **Description**                                                         |
| -- | ----------------- | ------------------ | ----------------------------------------------------------------------- |
| 15 | PRSADc            | Periodicity (PRSA) | PRSA capacity – magnitude of averaged oscillations                      |
| 16 | PRSADad           | Periodicity (PRSA) | PRSA amplitude difference                                               |
| 17 | PRSADos           | Periodicity (PRSA) | PRSA overall slope                                                      |
| 18 | PRSADsb           | Periodicity (PRSA) | PRSA slope before anchor                                                |
| 19 | PRSADsa           | Periodicity (PRSA) | PRSA slope after anchor                                                 |
| 20 | AC                | Periodicity        | Autocorrelation of SpO₂                                                 |
| 21 | PSDtotal          | Periodicity (PSD)  | Total power of the PSD                                                  |
| 22 | PSDband           | Periodicity (PSD)  | Power within 0.014–0.033 Hz band                                        |
| 23 | PSDratio          | Periodicity (PSD)  | PSDband / PSDtotal                                                      |
| 24 | PSDpeak           | Periodicity (PSD)  | Peak amplitude in 0.014–0.033 Hz band                                   |

In [13]:
prsa_class = PRSAMeasures(PRSA_Window=10, K_AC=2)
# Compute the biomarkers
print(f"{light_green}[DEBUG]{reset} Computing prsa periodicity features")
results_PRSA = prsa_class.compute(spo2)
print(f"{light_green}[DEBUG]{reset} Completed computing prsa periodicity features")

[DEBUG] Computing prsa periodicity features
[DEBUG] Completed computing prsa periodicity features


In [14]:
results_PRSA

PRSAResults(PRSAc=-0.4514939621781764, PRSAad=1.6899678342276871, PRSAos=-0.10670802749482837, PRSAsb=-0.055897002492454224, PRSAsa=-0.04225662349486735, AC=105522.71554495126)

In [15]:
psd_class = PSDMeasures()
# Compute the biomarkers
print(f"{light_green}[DEBUG]{reset} Computing psd periodicity features")
results_PSD = psd_class.compute(spo2)
print(f"{light_green}[DEBUG]{reset} Completed computing psd periodicity features")

[DEBUG] Computing psd periodicity features
[DEBUG] Completed computing psd periodicity features


In [16]:
results_PSD

PSDResults(PSD_total=0.02692750649280134, PSD_band=0.012600830193894658, PSD_ratio=0.4679538447892789, PSD_peak=0.00482352101229423)

### Desaturation Features

| #  | **Feature**       | **Category**       | **Description**                                                         |
| -- | ----------------- | ------------------ | ----------------------------------------------------------------------- |
| 25 | ODIx              | Desaturation       | Oxygen Desaturation Index (events/hour)                                 |
| 26 | DLµ               | Desaturation       | Mean duration of desaturation events                                    |
| 27 | DLσ               | Desaturation       | Std. of desaturation durations                                          |
| 28 | DDmaxµ            | Desaturation       | Mean desaturation depth                                                 |
| 29 | DDmaxσ            | Desaturation       | Std. of desaturation depth                                              |
| 30 | DD100µ            | Desaturation       | Mean depth using 100% as baseline                                       |
| 31 | DD100σ            | Desaturation       | Std. of depth using 100% as baseline                                    |
| 32 | DSµ               | Desaturation       | Mean slope of desaturations                                             |
| 33 | DSσ               | Desaturation       | Std. of slope of desaturations                                          |
| 34 | DAmaxµ            | Desaturation       | Mean area of desaturations (from max baseline)                          |
| 35 | DAmaxσ            | Desaturation       | Std. of area of desaturations                                           |
| 36 | DA100µ            | Desaturation       | Mean area under 100% baseline                                           |
| 37 | DA100σ            | Desaturation       | Std. of area under 100% baseline                                        |
| 38 | TDµ               | Desaturation       | Mean time between desaturation events                                   |
| 39 | TDσ               | Desaturation       | Std. of time between desaturation events                                |

**🧠 Why ODI (Oxygen Desaturation Index) is not used with hard thresholds (83, 85, 90):**

✅ 1. ODI is inherently a relative threshold-based feature

- ODI counts how many desaturations occur per hour based on a percentage drop from baseline (e.g., 3% or 5% drop from local baseline).

- The definition of a desaturation for ODI is:

`A ≥3% (or ≥4%, 5%) drop from the running or estimated baseline sustained for ≥10 seconds.`

❌ 2. Hard thresholds (like 85%, 90%) define absolute hypoxia, not dynamic changes

- They are used for hypoxic burden, not for detecting discrete events.

- You cannot define "ODI at 85%" because:

    - It's not about events, but about cumulative time below 85%.

    - If SpO₂ dips below 85%, you may be in a continuous hypoxic state, not experiencing discrete desaturations.

🧩 In Summary:

- ODI requires a % drop, not a fixed value.

- Hard thresholds (like 85%) are about being below a line, not dropping by a %.

- That’s why ODI is correctly only used with relative thresholds, and not computed or used with hard thresholds in your feature extraction.

In [17]:
# desat_class = DesaturationsMeasures(ODI_Threshold=3, threshold_method=DesatMethodEnum.Relative)
desat_class = DesaturationsMeasures(ODI_Threshold=3, hard_threshold=85, threshold_method=DesatMethodEnum.Hard)
# Compute the biomarkers with known desaturation locations
print(f"{light_green}[DEBUG]{reset} Computing desaturation features")
results_desat = desat_class.compute(spo2)
print(f"{light_green}[DEBUG]{reset} Completed computing desaturation features")

[DEBUG] Computing desaturation features
[DEBUG] Completed computing desaturation features


In [18]:
results_desat

DesaturationsMeasuresResults(ODI=0, DL_u=0, DL_sd=0, DA100_u=0, DA100_sd=0, DAmax_u=0, DAmax_sd=0, DD100_u=0, DD100_sd=0, DDmax_u=0, DDmax_sd=0, DS_u=0, DS_sd=0, TD_u=0, TD_sd=0, DL_a_u=0, DL_a_sd=0, DL_b_u=0, DL_b_sd=0, begin=[], end=[])

### Hypoxic Burden Features

| #  | **Feature**       | **Category**       | **Description**                                                         |
| -- | ----------------- | ------------------ | ----------------------------------------------------------------------- |
| 40 | PODx              | Hypoxic Burden     | Time under desaturation events normalized by recording time             |
| 41 | AODmax            | Hypoxic Burden     | Area under desaturation curve (max SpO₂ as baseline) normalized by time |
| 42 | AOD100            | Hypoxic Burden     | Area under 100% baseline normalized by time                             |
| 43 | CTx               | Hypoxic Burden     | Cumulative time under x% SpO₂                                           |
| 44 | CAx               | Hypoxic Burden     | Integral (area) of SpO₂ under x% level, normalized by time              |

In [19]:
hypoxic_class = HypoxicBurdenMeasures(results_desat.begin, results_desat.end, CT_Threshold=90, CA_Baseline=90)
# Compute the biomarkers
print(f"{light_green}[DEBUG]{reset} Computing burden features")
results_hypoxic = hypoxic_class.compute(spo2)
print(f"{light_green}[DEBUG]{reset} Completed computing burden features")

[DEBUG] Computing burden features
[DEBUG] Completed computing burden features


In [20]:
results_hypoxic

HypoxicBurdenMeasuresResults(CA=0.027718253968253958, CT=5.884920634920635, POD=0.0, AODmax=0.0, AOD100=0.0)

### Complexity Features

| #  | **Feature**       | **Category**       | **Description**                                                         |
| -- | ----------------- | ------------------ | ----------------------------------------------------------------------- |
| 10 | ApEn              | Complexity         | Approximate entropy – regularity of the time series                     |
| 11 | LZ                | Complexity         | Lempel–Ziv complexity – compressibility of the signal                   |
| 12 | CTMp              | Complexity         | Central tendency measure – deviation from a central tendency            |
| 13 | SampEn            | Complexity         | Sample entropy – complexity measure of time series                      |
| 14 | DFA               | Complexity         | Detrended Fluctuation Analysis – fractal scaling properties             |

**When you should compute these complexity features:**

- Your task benefits from complexity/irregularity analysis — e.g., detecting sleep apnea or physiological abnormalities often relies on entropy, Lempel-Ziv, DFA.

- You have manageable data size or enough compute power to run these calculations offline or batch mode.

- Your model or analysis improves significantly with these features — e.g., better accuracy, insight, or biomarker discovery.

- You can afford the computation time or optimize with tricks like windowing, caching, or JIT compiling.

**When you might skip or delay computing complexity features:**

- Your dataset is very large (millions of samples or many subjects) and time/resources are limited.

- Quick inference or real-time processing is required — these features are computationally expensive and may cause delays.

- You want a fast prototyping phase to explore simpler features first.

- Your baseline models work well without them; complexity features add little marginal gain.

- You can approximate or replace them with cheaper features (e.g., statistical moments, FFT features).

**Middle ground approaches:**

- Compute complexity features only on a subset of data or segments (windowed or event-triggered).

- Use lower embedding dimensions or smaller window sizes to speed up computations (at some loss of precision).

- Precompute and save these features offline for future use.

- Use hardware acceleration (like numba) or batch jobs.

### General Reasons Why calculating complexity features is slow

1. **Inefficient Algorithms**

    Many of the complexity measures (e.g., ApEn, SampEn) use **brute-force** or **nested loops** without vectorization. For large signals (e.g., thousands of samples), this results in:

* **O(N²)** or worse time complexity.
* Repeated comparisons and subarray creation.

2. **No Early Termination or Short-Circuiting**

    Functions like `comp_apen` and `comp_sampen` **compare every possible sub-sequence**, regardless of how little they differ — this is **computationally expensive** and not optimized.

3. **No Cython/Numba Acceleration**

    All calculations are pure Python/NumPy. Libraries like [`numba`](https://numba.pydata.org/) or `cython` would drastically reduce runtime but are not used.

---

**🔍 Detailed Breakdown**

🔴 `comp_apen()` + `__apen()`

* Time Complexity: **O(N²)**
* Reason: For each sub-sequence of size `m`, it compares with every other (`N - m + 1` × `N - m + 1` matrix of comparisons).
* Uses nested loops and logs — which are slow if done repeatedly.

🔴 `comp_sampen()`

* Time Complexity: **O(N²)**
* Reason: Similar brute-force search over sub-sequences of length `m` and `m+1`.
* Also has redundancy in calculations and no pruning of unpromising matches.

🔴 `comp_lz()`

* Time Complexity: **O(N²)** worst-case
* Reason: Creates substrings and keeps checking if they exist in a set. String operations in Python are **slow** and memory-intensive.

🟠 `comp_ctm()`

* Time Complexity: **O(N)**
* But the `sqrt()` inside the loop adds computational overhead.
* Not vectorized (looping through each triple of values).

🟡 `comp_dfa()`

* Time Complexity: **O(N)** (more precisely O(N/n) \* n for piecewise regression)
* Reason: Applies **linear regression** on **moving windows**.
* Uses `scipy.stats.linregress`, which is convenient but not optimized for repeated short-length calls.
